In [ ]:
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv

import os
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


# Load environment variables from the .env file
load_dotenv()

# 1. Setup the LLM
llm = ChatGroq(model="llama-3.3-70b-versatile")

# 2. Create a ChatPromptTemplate (Modern approach)
# We use MessagesPlaceholder to dynamically inject the chat history
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful naming consultant."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{product}"),
])

# 3. Create the Chain using LCEL (Pipe operator |)
chain = prompt | llm

print(llm , "Setup complete. You can now use the 'chain' object to generate names.")
KEY= os.getenv("GROKAI_API_KEY")
print("GROKAI_API_KEY:", KEY)

client=<groq.resources.chat.completions.Completions object at 0x1202a1ca0> async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x1202a2210> model_name='llama-3.3-70b-versatile' model_kwargs={} groq_api_key=SecretStr('**********') Setup complete. You can now use the 'chain' object to generate names.
GROKAI_API_KEY: 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


In [ ]:
import PyPDF2
from langchain_core.prompts import PromptTemplate


#Project: MCQ Generator


RESPONSE_JSON={
    "1": {
        "no": "1",
        "mcq": "multiple choice questions",
        "options":{
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        
        "correct": "correct answer"
    },
    
    "2": {
        "no": "2",
        "mcq": "multiple choice questions",
        "options":{
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        
        "correct": "correct answer"
    },
    
    "3": {
        "no": "3",
        "mcq": "multiple choice questions",
        "options":{
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        
        "correct": "correct answer"
    }
    
    
}


# 1st step design prompt using prompt template

template="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

quiz_generation_prompt = PromptTemplate(input_varibale =  ["text", "number", "grade", "tone", "response_json"],template = template)

print("Prompt template created successfully.")

chain = quiz_generation_prompt | llm
result = chain.invoke({"product": "drones"})


In [ ]:

template2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of teh question and give a complete analysis of the quiz if the students
will be able to unserstand the questions and answer them. Only use at max 50 words for complexity analysis. 
if the quiz is not at par with the cognitive and analytical abilities of the students,\
update tech quiz questions which needs to be changed  and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

quiz_evaluation_prompt2=PromptTemplate(input_variables=["subject", "quiz"], template=template)

print("Prompt template created successfully.")

reviewchain = quiz_evaluation_prompt2 | llm
result = chain.invoke({"product": "drones"})

In [14]:
from langchain_core.callbacks import BaseCallbackHandler

class GroqCostHandler(BaseCallbackHandler):
    def __init__(self):
        self.total_tokens = 0
        self.prompt_tokens = 0
        self.completion_tokens = 0
        self.successful_requests = 0
        
        # Pricing for Llama-3-70b (approximate Groq rates per 1M tokens)
        # You can update these rates based on current Groq pricing
        self.input_cost_per_1m = 0.59  # $0.59 per 1M input tokens
        self.output_cost_per_1m = 0.79 # $0.79 per 1M output tokens

    def on_llm_end(self, response, **kwargs):
        if response.llm_output and 'token_usage' in response.llm_output:
            usage = response.llm_output['token_usage']
            
            # Update counts
            self.prompt_tokens += usage.get('prompt_tokens', 0)
            self.completion_tokens += usage.get('completion_tokens', 0)
            self.total_tokens += usage.get('total_tokens', 0)
            self.successful_requests += 1

    def calculate_cost(self):
        input_cost = (self.prompt_tokens / 1_000_000) * self.input_cost_per_1m
        output_cost = (self.completion_tokens / 1_000_000) * self.output_cost_per_1m
        return input_cost + output_cost

    def print_report(self):
        cost = self.calculate_cost()
        print(f"\n--- Usage Report ---")
        print(f"Requests:      {self.successful_requests}")
        print(f"Input Tokens:  {self.prompt_tokens}")
        print(f"Output Tokens: {self.completion_tokens}")
        print(f"Total Tokens:  {self.total_tokens}")
        print(f"Total Cost:    ${cost:.6f}")
        print(f"--------------------\n")

In [16]:
import json
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
from langchain_core.callbacks import BaseCallbackHandler


# 1. Setup LLM
llm = ChatGroq(model="llama-3.3-70b-versatile")

# 2. Define the Response Format
RESPONSE_JSON = {
    "1": {
        "no": "1",
        "mcq": "multiple choice questions",
        "options": {"a": "...", "b": "...", "c": "...", "d": "..."},
        "correct": "correct answer"
    },
    "2": {
        "no": "2",
        "mcq": "...",
        "options": {"a": "...", "b": "...", "c": "...", "d": "..."},
        "correct": "..."
    },
    "3": {
        "no": "3",
        "mcq": "...",
        "options": {"a": "...", "b": "...", "c": "...", "d": "..."},
        "correct": "..."
    }
}

# ---------------------------------------------------------
# STEP 1: MCQ GENERATION CHAIN
# ---------------------------------------------------------

template1 = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs.

### RESPONSE_JSON
{response_json}
"""

# FIXED: 'input_variables' spelling corrected
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=template1
)

# Chain 1: Generates the Quiz text
# We add StrOutputParser() so the output is a string, not a message object
quiz_chain = quiz_generation_prompt | llm | StrOutputParser()

# ---------------------------------------------------------
# STEP 2: CRITIQUE/EVALUATION CHAIN
# ---------------------------------------------------------

template2 = """
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz if the students
will be able to unserstand the questions and answer them. Only use at max 50 words for complexity analysis. 
if the quiz is not at par with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities.

Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

# FIXED: used 'template=template2' (not template)
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject", "quiz"],
    template=template2
)

# Chain 2: Generates the Review
review_chain = quiz_evaluation_prompt | llm | StrOutputParser()

# ---------------------------------------------------------
# STEP 3: CONNECTING THEM (SEQUENTIAL CHAIN)
# ---------------------------------------------------------

# We use a dictionary to route arguments.
# 1. "quiz" gets generated by running quiz_chain.
# 2. "subject" is grabbed directly from the initial input using itemgetter.
overall_chain = (
    {"quiz": quiz_chain, "subject": itemgetter("subject")}
    | review_chain
)

# 2. Initialize the Handler
cb = GroqCostHandler()

# ---------------------------------------------------------
# EXECUTION
# ---------------------------------------------------------

# Sample text (Simulating your PDF text)
sample_text = """
Photosynthesis is the process used by plants, algae and certain bacteria to harness energy from sunlight and turn it into chemical energy.
There are two types of photosynthetic processes: oxygenic photosynthesis and anoxygenic photosynthesis.
The general equation for photosynthesis is: 6CO2 + 6H2O + Light Energy -> C6H12O6 + 6O2.
"""

# Run the chain
# Note: We pass 'json.dumps(RESPONSE_JSON)' to ensure the JSON guide is formatted as a string
result = overall_chain.invoke({
    "text": sample_text,
    "number": "3",
    "subject": "Biology",
    "tone": "Simple",
    "response_json": json.dumps(RESPONSE_JSON)
},config={'callbacks': [cb]}  )

print("### FINAL EVALUATION RESULT ###")
print(result)

# 4. Print the Cost Report
cb.print_report()


### FINAL EVALUATION RESULT ###
Complexity: Moderate to Challenging.

To better fit student abilities, I suggest rephrasing questions for clarity and simplicity:

1. What is photosynthesis used for in plants and some bacteria?
   a) Releasing energy
   b) Converting sunlight to chemical energy
   c) Breaking down glucose
   d) Producing water and carbon dioxide

2. How many kinds of photosynthesis are discussed?
   a) 1
   b) 2
   c) 3
   d) 4

3. What is produced during photosynthesis, besides glucose?
   a) Carbon dioxide
   b) Water
   c) Oxygen
   d) Light energy

--- Usage Report ---
Requests:      2
Input Tokens:  719
Output Tokens: 372
Total Tokens:  1091
Total Cost:    $0.000718
--------------------

